In [1]:
import pandas as pd
import numpy as np
import spacy
import preprocessing
from csv import reader 
pd.set_option('max_colwidth', 5000)

df = pd.read_csv('DB/Original_data.csv', index_col=[0])
#df = df.sample(frac=0.2)


In [2]:
#Check if there are null values 
df.isna().sum()

id         0
title      0
body       0
tags       0
answers    0
score      0
dtype: int64

In [3]:
#create column with answers aggregate by title of the questions
df2 = df 
func = lambda x: "\n".join(x)
df2 = df2.groupby('id')["answers"].agg([("answers",func)])



In [4]:
# concat answers for each post beacause there are a record with question duplicated for each answer to that question 


grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg("sum","score")

grouped_df = pd.DataFrame(grouped)
grouped_df = pd.merge(grouped_df, df2, left_on='id', right_on='id', how='left')
grouped_df



id  \
0            742   
1            773   
2           1829   
3           2933   
4           5102   
...          ...   
297600  69060217   
297601  69060327   
297602  69060696   
297603  69060948   
297604  69061001   

                                                                                            title  \
0                                                                           Class views in Django   
1                                                               How do I use itertools.groupby()?   
2       How do I make a menu that does not require the user to press [enter] to make a selection?   
3                                Create a directly-executable cross-platform GUI app using Python   
4                                         How do you set up Python scripts to work in Apache 2.0?   
...                                                                                           ...   
297600                                QLineSeries can't append the QDateTime().toMsecsSincEpoch()   
297601                                                 Problem with the print statement in Python   
297602                                        How do I extract two dates from a string in python?   
297603                                                                   Python Input and Output?   
297604                                                  Dutch National Flag Python Implementation   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  body  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       <p><a href="http://www.djangoproject.com/" rel="noreferrer">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.</p>\n\n<p>For exa

In [5]:
#Preprocessing Part
#Answers
#Removing tags


answers = grouped_df["answers"]
preprocessing.remove_tags(answers)



#Clearing text 
answers_processed = answers.apply(lambda x: preprocessing.clear_text(x,"w2v"))


In [6]:
answers_processed.head()
answers_processed.isna().sum() 

0

In [7]:
#Questions
#Merge title with body 
questions = grouped_df["title"] + grouped_df["body"]
preprocessing.remove_tags(questions)
questions

0         Class views in Django  view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.    For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc...    One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?  
1                          How do I use itertools.groupby ? I haven't been able to find an understandable explanation of how to actually use Python's  function.  What I'm trying to do is this:     Take a list - in this case, the children of an objectified  element   Divide it into groups based on some criteria   Then later iterate over each of these groups separately.     I've revi

In [8]:
#Clearing text 
questions_processed = questions.apply(lambda x: preprocessing.clear_text(x,"w2v"))
questions_processed

0         class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea 
1                             use itertools groupby able find understandable explanation actually use python function trying take list case children objectified element divide groups based criteria later iterate groups separately reviewed trouble trying apply beyond simple list numbers use another technique using pointers good quot prerequisite quot reading would also appreciated 
2                                                                                                                                                      make menu require user press enter make selection got menu python part easy using

In [9]:
#Create a column to add in db with question title, question body and all the answers processed, to train later the W2V embeddings
post_corpus = questions_processed + answers_processed
grouped_df.drop("answers", axis=1, inplace=True)
grouped_df.drop("body", axis=1, inplace=True)
grouped_df["post_corpus"] = post_corpus
grouped_df["questions"] = questions
grouped_df

,id,title,tags,score,post_corpus,questions
0,742,Class views in Django,python|django|views|oop,75,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance created used generic view classes defining instance class callable really like django generic views allow customization keyword arguments oo generic views much fine grained customization via subclassing lets repeat lot less posted code real downside see proliferation internal method calls may impact performance somewhat think much concern rare python code execution would performance bottleneck web app update django class based update fwiw changed opinion class based views since answer written used extensively couple projects feel tend lead code satisfyingly dry write hard read maintain later functionality spread across many different places subclasses dependent every implementation detail superclasses mixins feel view decorators better answer decomposing view code want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place sounds like trying combine things combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan unless want something little complex using generic views way go far powerful name implies displaying model data generic views job use django generic views easily achieve desired functionality thorough django generic views always create class override function point url file instance class take look class see done generic views usually way go ultimately free handle urls however want formwizard things class based way apps restful apis basically url given bunch variables place provide callable callable provide completely standard way provide function ultimately django puts restrictions agree examples would good formwizard probably place start though simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?"
1,773,How do I use itertools.groupby()?,python|itertools,1081,use itertools groupby able find understandable explanation actually use python function trying take list case children objectified element divide groups based criteria later iterate groups separately reviewed trouble trying apply beyond simple list numbers use another technique using pointers good quot prerequisite quot reading would also appreciated neato trick groupby run length encoding one line 

In [10]:
#Filter out the tags, selecting only the 20 most common for better processing 
# Convert raw text data of tags into lists
grouped_df["tags"] = grouped_df["tags"].apply(lambda x: x.split('|'))   

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}

#i = 0 

for tags in grouped_df["tags"]:
    for tag in tags:
        #Remove tags python, python2.7 e python3 for further processing 
        if "python" not in tag :
            
            if tag not in tag_freq_dict:
                tag_freq_dict[tag] = 0
            else:
                tag_freq_dict[tag] += 1
        else:
            tags.remove(tag)
    '''
    #If a record has less than 3 tags eliminate it for better further processing (training of fastText)
    if len(tags) < 3:
        grouped_df.drop(i)
    i = i +1
    '''
grouped_df["tags"]


0                                              [django, views, oop]
1                                                       [itertools]
2                                                                []
3         [user-interface, deployment, tkinter, release-management]
4                                                 [apache, apache2]
                                    ...                            
297600                                            [pyside6, qchart]
297601                                                           []
297602                           [regex, web-scraping, re, findall]
297603                                              [input, output]
297604                  [if-statement, dutch-national-flag-problem]
Name: tags, Length: 297605, dtype: object

In [11]:
#Selecting the most common number of tags in our database 
import heapq
most_common_tags = heapq.nlargest(20, tag_freq_dict, key=tag_freq_dict.get)
most_common_tags

['dataframe',
 'numpy',
 'pandas',
 'dictionary',
 'django',
 'matplotlib',
 'list',
 'beautifulsoup',
 'tkinter',
 'flask',
 'keras',
 'csv',
 'javascript',
 'django-models',
 'web-scraping',
 'scipy',
 'scikit-learn',
 'for-loop',
 'datetime',
 'django-rest-framework']

In [12]:
#Select only the data with at least one of the most common tags 
final_indices = []
for i,tags in enumerate(grouped_df["tags"].values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>0:   # The minimum length for common tags should be 1
        final_indices.append(i)

final_data = grouped_df.iloc[final_indices]

final_data 

,id,title,tags,score,post_corpus,questions
0,742,Class views in Django,"[django, views, oop]",75,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance created used generic view classes defining instance class callable really like django generic views allow customization keyword arguments oo generic views much fine grained customization via subclassing lets repeat lot less posted code real downside see proliferation internal method calls may impact performance somewhat think much concern rare python code execution would performance bottleneck web app update django class based update fwiw changed opinion class based views since answer written used extensively couple projects feel tend lead code satisfyingly dry write hard read maintain later functionality spread across many different places subclasses dependent every implementation detail superclasses mixins feel view decorators better answer decomposing view code want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place sounds like trying combine things combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan unless want something little complex using generic views way go far powerful name implies displaying model data generic views job use django generic views easily achieve desired functionality thorough django generic views always create class override function point url file instance class take look class see done generic views usually way go ultimately free handle urls however want formwizard things class based way apps restful apis basically url given bunch variables place provide callable callable provide completely standard way provide function ultimately django puts restrictions agree examples would good formwizard probably place start though simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?"
3,2933,Create a directly-executable cross-platform GUI app using Python,"[user-interface, deployment, tkinter, release-management]",410,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please pysimplegui wraps tkinter works python also runs qt wxpython web browser

In [13]:
#Create a column only for the processed title of the questions 
final_data['processed_title'] = final_data.title.apply(lambda x: preprocessing.clear_text(x,"w2v"))

/var/folders/7j/b9v_1c594f74fpz8g452mpfr0000gn/T/ipykernel_1709/2150014301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['processed_title'] = final_data.title.apply(lambda x: preprocessing.clear_text(x,"w2v"))


In [14]:
# Normalize numeric data for the scores
final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())

/var/folders/7j/b9v_1c594f74fpz8g452mpfr0000gn/T/ipykernel_1709/3151826978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())


In [15]:
# Combine the lists back into text data
final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))

final_data

/var/folders/7j/b9v_1c594f74fpz8g452mpfr0000gn/T/ipykernel_1709/3494508092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))


,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.006907,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance created used generic view classes defining instance class callable really like django generic views allow customization keyword arguments oo generic views much fine grained customization via subclassing lets repeat lot less posted code real downside see proliferation internal method calls may impact performance somewhat think much concern rare python code execution would performance bottleneck web app update django class based update fwiw changed opinion class based views since answer written used extensively couple projects feel tend lead code satisfyingly dry write hard read maintain later functionality spread across many different places subclasses dependent every implementation detail superclasses mixins feel view decorators better answer decomposing view code want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place sounds like trying combine things combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan unless want something little complex using generic views way go far powerful name implies displaying model data generic views job use django generic views easily achieve desired functionality thorough django generic views always create class override function point url file instance class take look class see done generic views usually way go ultimately free handle urls however want formwizard things class based way apps restful apis basically url given bunch variables place provide callable callable provide completely standard way provide function ultimately django puts restrictions agree examples would good formwizard probably place start though simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?",class views django
3,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.040013,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please pysimplegui wraps tkinter works python 

In [16]:
#Check if the final data has some null values 
final_data.isna().sum()

final_data.dropna()
#final_data = final_data[final_data['post_corpus'].notna()]

,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.006907,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate view class works checking first argument must actual http request would nonsense attept instantiate view class instance created used generic view classes defining instance class callable really like django generic views allow customization keyword arguments oo generic views much fine grained customization via subclassing lets repeat lot less posted code real downside see proliferation internal method calls may impact performance somewhat think much concern rare python code execution would performance bottleneck web app update django class based update fwiw changed opinion class based views since answer written used extensively couple projects feel tend lead code satisfyingly dry write hard read maintain later functionality spread across many different places subclasses dependent every implementation detail superclasses mixins feel view decorators better answer decomposing view code want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place sounds like trying combine things combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan unless want something little complex using generic views way go far powerful name implies displaying model data generic views job use django generic views easily achieve desired functionality thorough django generic views always create class override function point url file instance class take look class see done generic views usually way go ultimately free handle urls however want formwizard things class based way apps restful apis basically url given bunch variables place provide callable callable provide completely standard way provide function ultimately django puts restrictions agree examples would good formwizard probably place start though simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach. For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc... One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea?",class views django
3,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.040013,create directly executable cross platform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please pysimplegui wraps tkinter works python 

In [17]:
# Save the data
final_data.to_csv('DB/Preprocessed_data.csv', index=False)